# Module 5: Beyond Query Language — DDL, DML, and DCL

This notebook introduces the SQL sublanguages beyond querying:  
- DDL (Data Definition Language)  
- DML (Data Manipulation Language)  
- DCL (Data Control Language)

We will explain each concept and demonstrate its use with SQL commands against the `farmers_market` database.

In [47]:
import mysql.connector
import pandas as pd

# Establish connection to local MySQL server
conn = mysql.connector.connect(
    host="localhost",
    user="root",  # Never use root in production
    password="William2025!!",  # Replace with secure credentials
    database="farmers_market"
)

cursor = conn.cursor()
print(f"Successfully connected to {conn.database}!")

Successfully connected to farmers_market!


## SQL Sublanguages Overview

SQL is divided into several sublanguages:

- **DQL** (Data Query Language): `SELECT` — used for reading data (covered in Modules 3 & 4)
- **DDL** (Data Definition Language): `CREATE`, `DROP`, `ALTER` — defines structure
- **DML** (Data Manipulation Language): `INSERT`, `UPDATE`, `DELETE` — modifies data
- **DCL** (Data Control Language): `GRANT`, `REVOKE` — controls access to data

## DDL: CREATE VIEW

## Difference Between Tables and Views

**Tables:**
- Physically store data in the database.
- Can be modified directly with INSERT, UPDATE, and DELETE statements.
- Used for persistent storage of structured data.

**Views:**
- Are virtual tables based on the result of a SQL query.
- Do not store data themselves; they display data from one or more tables.
- Cannot always be modified directly (depends on the view definition and database rules).
- Useful for simplifying complex queries, providing security, and presenting data in a specific format.


We define a new view using the `CREATE VIEW` command.
The example below creates a view based on a multi-table join (booth + vendor assignments + vendor).



In [21]:
# Create a view for vendor booth assignments
query = '''
CREATE OR REPLACE VIEW farmers_market.booth_date_vendors AS
SELECT
    b.booth_number,
    b.booth_type,
    vba.market_date,
    v.vendor_id,
    v.vendor_name
FROM farmers_market.booth AS b
LEFT JOIN farmers_market.vendor_booth_assignments AS vba ON b.booth_number = vba.booth_number
LEFT JOIN farmers_market.vendor AS v ON v.vendor_id = vba.vendor_id;
'''

cursor = conn.cursor()
cursor.execute(query)
cursor.close()
print("View 'booth_date_vendors' created successfully.")

View 'booth_date_vendors' created successfully.


### Retrive Data from View

In [22]:
# Retrieve data from the view
query = "SELECT * FROM farmers_market.booth_date_vendors LIMIT 10;"
cursor = conn.cursor()
cursor.execute(query)
df_view = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
cursor.close()
df_view

,booth_number,booth_type,market_date,vendor_id,vendor_name
0,1,Standard,2019-04-03,3,Mountain View Vegetables
1,1,Standard,2019-04-06,3,Mountain View Vegetables
2,1,Standard,2019-04-10,3,Mountain View Vegetables
3,1,Standard,2019-04-13,3,Mountain View Vegetables
4,1,Standard,2019-04-17,3,Mountain View Vegetables
5,1,Standard,2019-04-20,3,Mountain View Vegetables
6,1,Standard,2019-04-24,3,Mountain View Vegetables
7,1,Standard,2019-04-27,3,Mountain View Vegetables
8,1,Standard,2019-05-01,3,Mountain View Vegetables
9,1,Standard,2019-05-04,3,Mountain View Vegetables


## DDL: CREATE TABLE

We define a new table using the `CREATE TABLE` command.
The example below creates a view based on a multi-table join (booth + vendor assignments + vendor).

In [26]:
# Create the volunteers table with three fields
query = '''
CREATE TABLE IF NOT EXISTS farmers_market.volunteers (
    volunteer_id INT AUTO_INCREMENT PRIMARY KEY,
    volunteer_name VARCHAR(50) NOT NULL,
    volunteer_last_name VARCHAR(50) NOT NULL
);
'''

cursor = conn.cursor()
cursor.execute(query)
conn.commit()
cursor.close()
print("Table 'volunteers' created successfully.")

Table 'volunteers' created successfully.


## DDL: DROP TABLE

The `DROP` statement permanently removes a table.  
 
## RESTRICT vs CASCADE in DROP Statements

- **RESTRICT:** Prevents the table or view from being dropped if it is referenced by another object (such as a foreign key or another view). The drop operation will fail if dependencies exist.
- **CASCADE:** Automatically drops the table or view and all dependent objects (such as foreign key constraints, views, or other related objects). Use with caution, as this can remove more than just the specified object.

**Example:**
- `DROP TABLE my_table RESTRICT;` — Fails if other tables depend on `my_table`.
- `DROP TABLE my_table CASCADE;` — Drops `my_table` and any dependent objects.

In [25]:
# Example: Drop table with RESTRICT or CASCADE (simulated, not executed)
drop_query = "DROP TABLE IF EXISTS farmers_market.volunteers RESTRICT;"


cursor = conn.cursor()
cursor.execute(drop_query)   
conn.commit()
cursor.close()




print("DROP TABLE (RESTRICT option).")

DROP TABLE (RESTRICT option).


## DDL: ALTER TABLE

The `ALTER TABLE` command allows you to modify the structure of existing tables.

In [27]:
# Example: Add columns to the volunteers table
alter_query = '''
ALTER TABLE farmers_market.volunteers
    ADD COLUMN email VARCHAR(100),
    ADD COLUMN number CHAR(10) CHECK (number REGEXP '^[0-9]{10}$');
'''

cursor = conn.cursor()
cursor.execute(alter_query)
conn.commit()
cursor.close()
print("Columns 'email' and 'number' added to the 'volunteers' table successfully.")

Columns 'email' and 'number' added to the 'volunteers' table successfully.


### Now let's focus on DML (Data Manipulation Language)

## DML: INSERT

Use `INSERT INTO` to add a new row of data.

In [28]:
# Example: Insert new volunteer
insert_query = '''
INSERT INTO farmers_market.volunteers (volunteer_name, volunteer_last_name, email, number)
VALUES ('John', 'Doe', 'john.doe@example.com', '1234567890');
'''
cursor = conn.cursor()
cursor.execute(insert_query)
conn.commit()
cursor.close()
print("Inserted new volunteer: 'John Doe'")

Inserted new volunteer: 'John Doe'


## DML: UPDATE

Use `UPDATE` to modify data in existing records.

In [14]:
# Example: Update a volunteer's email and number
update_query = '''
UPDATE farmers_market.volunteers
SET email = 'john.newemail@example.com', number = '0987654321'
WHERE volunteer_name = 'John' AND volunteer_last_name = 'Doe';
'''
cursor = conn.cursor()
cursor.execute(update_query)
conn.commit()
cursor.close()
print("Updated volunteer's email and number.")

Updated volunteer's email and number.


## DML: DELETE

Use `DELETE FROM` to remove rows from a table.

In [29]:
# Example: Delete a volunteer by name
delete_query = '''
DELETE FROM farmers_market.volunteers
WHERE volunteer_name = 'John' AND volunteer_last_name = 'Doe';
'''
cursor = conn.cursor()
cursor.execute(delete_query)
conn.commit()
cursor.close()
print("Deleted volunteer: 'John Doe'")

Deleted volunteer: 'John Doe'


## Creating a user using SQL Command


## DCL: GRANT and REVOKE

The `GRANT` statement provides access privileges to users, while `REVOKE` removes them.

```sql
GRANT SELECT ON farmers_market.product TO 'analyst'@'localhost';
REVOKE SELECT ON farmers_market.product FROM 'analyst'@'localhost';
```

**Note:** These commands require administrative privileges and are typically executed by DBAs.

In [48]:

# Define the queries
drop_user_query = "DROP USER IF EXISTS 'new_user'@'localhost';"
create_user_query = "CREATE USER 'new_user'@'localhost' IDENTIFIED BY 'new_password';"
grant_privileges_query = "GRANT SELECT ON farmers_market.volunteers TO 'new_user'@'localhost';"
flush_privileges_query = "FLUSH PRIVILEGES;"

cursor = conn.cursor()
# Execute the queries
cursor.execute(drop_user_query)
cursor.execute(create_user_query)
cursor.execute(grant_privileges_query)
# cursor.execute(flush_privileges_query)

print("User created, privileges granted, and changes applied successfully.")

cursor.close()
conn.close()

User created, privileges granted, and changes applied successfully.
